In [ ]:
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import pandas as pd
import xlwt
import os
import glob
o = 8.854187817*10**(-12)
def realimag(array):
    return np.array([(x.real, -x.imag) for x in array])

def func(x,p):
    s,u,sigma,t=p
    d = complex(0,1)
    o = 8.854187817*10**(-12)
    return realimag(u+(s-u)/(1+np.dot(d,np.dot(x,t)))-np.dot(d,np.divide(sigma,np.dot(x,o))))

def conduct_loss_result(x,p):
    s,u,sigma,t=p
    o=8.854187817*10**(-12)
    return np.divide(sigma,np.dot(x,o))

def relax_loss_result(x,p):
    s,u,sigma,t=p
    return np.dot(np.divide(s-u,(1+np.dot(x,t)**2)),np.dot(x,t))

def residuals(p,y,x):
    return (realimag(np.array(y)) - func(x,p)).flatten()

def process_file(file_path, output_dir):
    p0 = [45,2,1,10**-11]
    data = pd.read_excel(file_path, header=None, skiprows=1)
    table = data

    frequency_list = []
    ydata2_mean_list = []
    ydata1_mean_list = []
    fs_list = []
    fu_list = []
    fsig_list = []
    ft_list = []
    fconduct_loss_list = []
    frelax_loss_list = []
    fydata2_mean_list = []
    fydata1_mean_list = []

    for i in range(1,201):
        end=i*2
        start=end-2
        xdata = table.iloc[start:end, 0].values
        frequency = np.mean(xdata)
        # 只处理0.11-1.5THz范围内的数据
        if 0.11 <= frequency <= 1.5:
            xdata1 = 2 * np.pi * xdata * 1e12
            xdata = xdata1
            ydata_1 = table.iloc[start:end, 2].values
            ydata_2 = table.iloc[start:end, 1].values
            ydata = []
            ydata1_mean = np.mean(ydata_1)
            ydata2_mean = np.mean(ydata_2)

            for j in range(2):
                ydata.append(complex(ydata_2[j], -ydata_1[j]))

            plsq = least_squares(residuals, p0, bounds=([0,0,0,0.8*10**-13],[65,10,1000,10**-10]),args=(ydata, xdata),max_nfev=100000)


            fs = plsq.x[0]
            fu = plsq.x[1]
            fsig = plsq.x[2]
            ft = plsq.x[3]
            fconduct_loss = np.mean(conduct_loss_result(xdata, plsq.x))
            frelax_loss = np.mean(relax_loss_result(xdata, plsq.x))
            fydata2 = fu + np.divide((fs - fu), (1 + (np.dot(xdata, ft) ** 2)))
            fydata1 = np.dot(np.divide((fs - fu), (1 + (np.dot(xdata, ft) ** 2))), np.dot(xdata, ft)) + np.divide(fsig, np.dot(xdata, o))

            frequency_list.append(frequency)
            fs_list.append(fs)
            fu_list.append(fu)
            fsig_list.append(fsig)
            ft_list.append(ft)
            fconduct_loss_list.append(fconduct_loss)
            frelax_loss_list.append(frelax_loss)
            fydata2_mean_list.append(np.mean(fydata2))
            fydata1_mean_list.append(np.mean(fydata1))
            ydata2_mean_list.append(ydata2_mean)
            ydata1_mean_list.append(ydata1_mean)

    # 创建Excel文件并保存数据
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    col = ('frequency','conduct_loss','relax_loss','conductivity','static dielectric constant','optical dielectric constant','relax_time','ydata2','ydata1','fydata2','fydata1')
    for i in range(0,11):
        sheet1.write(0,i,col[i])
    for i in range(len(frequency_list)):
        sheet1.write(i+1,0,frequency_list[i])
        sheet1.write(i+1,1,fconduct_loss_list[i])
        sheet1.write(i+1,2,frelax_loss_list[i])
        sheet1.write(i+1,3,fsig_list[i])
        sheet1.write(i+1,4,fs_list[i])
        sheet1.write(i+1,5,fu_list[i])
        sheet1.write(i+1,6,ft_list[i])
        sheet1.write(i+1,7,ydata2_mean_list[i])
        sheet1.write(i+1,8,ydata1_mean_list[i])
        sheet1.write(i+1,9,fydata2_mean_list[i])
        sheet1.write(i+1,10,fydata1_mean_list[i])

    # 获取样品信号（假设是文件名的一部分）
    sample_signal = os.path.basename(file_path).split('.')[0]

    # 设置保存文件名
    save_filename = f"{sample_signal}_analysis_results.xls"
    save_path = os.path.join(output_dir, save_filename)

    # 保存 Excel 文件
    f.save(save_path)
    print(f"Excel file saved: {save_path}")

    # 绘制并保存图像
    plt.figure(figsize=(10, 6))
    plt.plot(frequency_list, fconduct_loss_list, 'b-', label='Conduction Loss')
    plt.plot(frequency_list, frelax_loss_list, 'r-', label='Relaxation Loss')
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Loss')
    plt.xlim(0.11, 1.5)  # 设置x轴范围
    plt.title(f'{sample_signal}: Frequency vs Losses')
    plt.legend()
    plt.grid(True)
    loss_plot_path = os.path.join(output_dir, f"{sample_signal}_frequency_vs_losses.png")
    plt.savefig(loss_plot_path)
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(frequency_list, fsig_list, 'g-')
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Conductivity')
    plt.title(f'{sample_signal}: Frequency vs Conductivity')
    plt.grid(True)
    plt.xlim(0.11, 1.5)  # 设置x轴范围
    conductivity_plot_path = os.path.join(output_dir, f"{sample_signal}_frequency_vs_conductivity.png")
    plt.savefig(conductivity_plot_path)
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(frequency_list,ydata2_mean_list, '^', label=u"eps'_exp", linewidth=3)
    plt.plot(frequency_list,fydata2_mean_list, 'o', label=u"eps'_fit", linewidth=1)
    plt.title(u" ")
    plt.legend()
    plt.xlabel(u"Frequency")
    plt.ylabel(u"eps'")
    plt.xlim(0.11, 1.5)  # 设置x轴范围
    image_plot_path = os.path.join(output_dir, f"{sample_signal}_frequency_vs_image.png")
    plt.savefig(image_plot_path)
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(frequency_list,ydata1_mean_list, '*', label=u"eps''_exp", linewidth=3)
    plt.plot(frequency_list,fydata1_mean_list, 'o', label=u"eps''_fit", linewidth=1)
    plt.title(u" ")
    plt.legend()
    plt.xlabel(u"Frequency")
    plt.xlim(0.11, 1.5)  # 设置x轴范围
    plt.ylabel(u"eps''")
    real_plot_path = os.path.join(output_dir, f"{sample_signal}_frequency_vs_real.png")
    plt.savefig(real_plot_path)
    plt.close()

    print(f"Plots saved for {sample_signal}")

# 设置输入和输出目录
input_dir = r'D:\张雨林\导电力学协同优化\原始数据\太赫兹吸收测试\1mm处理后结果文件'
output_dir = r'D:\张雨林\导电力学协同优化\原始数据\太赫兹吸收测试\1mm处理后结果文件\损耗分离结果'

# 确保输出目录存在
os.makedirs(output_dir, exist_ok=True)

# 使用通配符匹配所有符合条件的文件
file_pattern = "*_Electromagnetic_Properties.xlsx"
file_paths = glob.glob(os.path.join(input_dir, file_pattern))

for file_path in file_paths:
    file_name = os.path.basename(file_path)
    print(f"Processing file: {file_name}")
    
    # 提取文件名前缀（序号和字母）
    prefix = file_name.split('.pulse_Electromagnetic_Properties.xlsx')[0]
    
    # 调用处理函数，传入文件路径、输出目录和文件前缀
    process_file(file_path, output_dir)

if not file_paths:
    print(f"No matching files found in: {input_dir}")
